In [2]:
#%pip install pandas


In [1]:

import pandas as pd

import os
import glob

from multiprocessing import Pool

from datetime import date
from time import monotonic



In [2]:
# Loading into dataframe - solution 1 - 35s

data_dir = "inp" 
columns_list = ['var_symbol', 'creation_date', 'due_date', 'amount', 'VAT', 'payment_type', 'is_paid']


data = pd.DataFrame(columns=columns_list)

for root, dirs, files in os.walk(data_dir):
    for name in files:
        
        invoice = pd.read_csv(os.path.join(data_dir, name), sep=';', names=columns_list)        # this is the slowest       
        invoice["id"] = name.removesuffix(".csv")                                               # 10% time

        data = pd.concat([data, invoice])                                                       # 10% time


data["creation_date"] = pd.to_datetime(data["creation_date"])
data["due_date"] = pd.to_datetime(data["due_date"])                             # for better datetime handling



data.reset_index(drop=True, inplace=True)
data.head()


,var_symbol,creation_date,due_date,amount,VAT,payment_type,is_paid,id
0,104251,2022-01-01 01:38:00,2022-01-15 01:38:00,13945,20,pay_by_link,1,1640997480
1,100734,2022-01-01 01:52:00,2022-01-15 01:52:00,6478,10,backward_invocing,0,1640998320
2,102016,2022-01-01 03:47:00,2022-01-15 03:47:00,9511,20,pay_by_link,1,1641005220
3,103654,2022-01-01 04:04:00,2022-01-15 04:04:00,13366,10,backward_invocing,0,1641006240
4,101035,2022-01-01 04:26:00,2022-01-15 04:26:00,14714,10,pay_by_link,1,1641007560


In [3]:
# Loading into dataframe - solution 2 - 20s


data_dir = "inp" 
columns_list = ['var_symbol', 'creation_date', 'due_date', 'amount', 'VAT', 'payment_type', 'is_paid']

start_time = monotonic()




all_files = glob.glob(os.path.join(data_dir, "*.csv"))                          # list

print("files done:", (monotonic() - start_time) * 1000.0, "ms")

invoices = [pd.read_csv(f, sep=';', names=columns_list) for f in all_files]     # the slowest part, bound by CPU

print("loading done:", (monotonic() - start_time) * 1000.0, "ms")

data = pd.concat(invoices, ignore_index=True)

print("concat done:", (monotonic() - start_time) * 1000.0, "ms")

data = data.assign(ID = all_files)                                              # adds ID column from filename
data.ID = data.ID.str.replace(r"\D+", "", regex=True)                           # removes dir and .csv part

print("assign done:", (monotonic() - start_time) * 1000.0, "ms")

data["creation_date"] = pd.to_datetime(data["creation_date"])
data["due_date"] = pd.to_datetime(data["due_date"])                             # for better datetime handling

data.set_index('ID', inplace=True)

data.head()


files done: 125.0 ms
loading done: 18077.99999997951 ms
concat done: 20655.99999995902 ms
assign done: 20670.99999997299 ms


,var_symbol,creation_date,due_date,amount,VAT,payment_type,is_paid
ID,,,,,,,
1640997480,104251,2022-01-01 01:38:00,2022-01-15 01:38:00,13945,20,pay_by_link,1
1640998320,100734,2022-01-01 01:52:00,2022-01-15 01:52:00,6478,10,backward_invocing,0
1641005220,102016,2022-01-01 03:47:00,2022-01-15 03:47:00,9511,20,pay_by_link,1
1641006240,103654,2022-01-01 04:04:00,2022-01-15 04:04:00,13366,10,backward_invocing,0
1641007560,101035,2022-01-01 04:26:00,2022-01-15 04:26:00,14714,10,pay_by_link,1


In [ ]:
# Loading into dataframe - solution 3 - multiprocessing - 10s - only works with .py script - see multi.py 


data_dir = "inp" 
columns_list = ['var_symbol', 'creation_date', 'due_date', 'amount', 'VAT', 'payment_type', 'is_paid']

start_time = monotonic()

n_workers = 4



def load_func(files : list[str]):
    """
    Input - list of str, that contains names and dir of csv files
    Output - merged dataframe that also contains name (ID) of files
    """
    
    columns_list = ['var_symbol', 'creation_date', 'due_date', 'amount', 'VAT', 'payment_type', 'is_paid']
    
    invoices = [pd.read_csv(f, sep=';', names=columns_list) for f in files] 
    invoices = pd.concat(invoices, ignore_index=True).assign(ID = files)            # adds ID column from filename           
    invoices.ID = invoices.ID.str.replace(r"\D+", "", regex=True)                   # removes dir and .csv part from filename
    
    return invoices


def divide_into_equal_chunks(lst : list, n_chunks = 4) -> list:
    chunk_size = len(lst) // n_chunks + 1
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]
        
        


all_files = glob.glob(os.path.join(data_dir, "*.csv"))                          # list
chunks = divide_into_equal_chunks(all_files, n_chunks=4)                               # dividing all the files into chunks that run in parallel

print("files done:", (monotonic() - start_time) * 1000.0, "ms")

with Pool(processes=n_workers) as p:
    invoices = p.map(load_func, chunks)
    
print("processing done:", (monotonic() - start_time) * 1000.0, "ms")

data = pd.concat(invoices)                                                      # merging chunks together

print("merging done:", (monotonic() - start_time) * 1000.0, "ms")


data["creation_date"] = pd.to_datetime(data["creation_date"])
data["due_date"] = pd.to_datetime(data["due_date"])                             # for better datetime handling

data.set_index('ID', inplace=True)


data.head()


In [4]:
# 1. List of invoices created last month
output_dir = "output"

try:
    os.mkdir(output_dir)
except Exception as e:
    print("Cannot create output directory:", e)



last_month = (pd.Timestamp.utcnow() - pd.DateOffset(months=1)).to_period("M")                   # UTC instead of local, get year and month
last_month_invoices = data[data.creation_date.dt.to_period("M")  == last_month]

last_month_invoices.to_csv(os.path.join(output_dir, "last_month_invoices.csv"))                          # save to csv

last_month_invoices.head(1000)


Cannot create output directory: [WinError 183] Nelze vytvořit soubor, který již existuje: 'output'


C:\Users\Nomad\AppData\Local\Temp\ipykernel_19592\3520720787.py:11: UserWarning: Converting to Period representation will drop timezone information.
  last_month = (pd.Timestamp.utcnow() - pd.DateOffset(months=1)).to_period("M")                   # UTC instead of local, get year and month


,var_symbol,creation_date,due_date,amount,VAT,payment_type,is_paid
ID,,,,,,,
1661983200,100243,2022-09-01 00:00:00,2022-09-15 00:00:00,7302,10,backward_invocing,1
1661985840,100546,2022-09-01 00:44:00,2022-09-15 00:44:00,3891,0,card,0
1661989620,103584,2022-09-01 01:47:00,2022-09-15 01:47:00,4147,20,pay_by_link,0
1661997420,100984,2022-09-01 03:57:00,2022-09-15 03:57:00,11031,10,backward_invocing,0
1661999460,104179,2022-09-01 04:31:00,2022-09-15 04:31:00,7393,20,backward_invocing,1
...,...,...,...,...,...,...,...
1664378460,101805,2022-09-28 17:21:00,2022-10-12 17:21:00,10472,10,backward_invocing,0
1664382300,101900,2022-09-28 18:25:00,2022-10-12 18:25:00,1081,0,backward_invocing,1
1664385300,102717,2022-09-28 19:15:00,2022-10-12 19:15:00,12913,20,card,0


In [5]:
# 2. The total amount of created invoices per month (without VAT and with VAT)


VAT_per_month = data.resample("M", on="creation_date")["VAT"].value_counts().sort_index().unstack('VAT').fillna(0)     #group, count VAT values, align
VAT_per_month.index = VAT_per_month.index.to_period("M")               # don't need days

VAT_per_month.to_csv(os.path.join(output_dir, "VAT_per_month.csv"))

VAT_per_month.head(20)


VAT,0,10,20
creation_date,,,
2022-01,141,156,126
2022-02,122,140,134
2022-03,119,121,155
2022-04,148,145,143
2022-05,150,147,135
2022-06,171,136,143
2022-07,139,117,142
2022-08,125,135,130
2022-09,128,148,119


In [6]:
# 3. Total amount by Payment type


total_by_type = data["payment_type"].value_counts().to_frame().reset_index()
total_by_type.columns = ["Payment type", "Count"]

total_by_type.to_csv(os.path.join(output_dir, "total_by_payment_type.csv"), index=False)

total_by_type.head(20)


,Payment type,Count
0,backward_invocing,1672
1,pay_by_link,1661
2,card,1637


In [7]:
# 4. Find unpaid invoices for today

created_before_now = data.creation_date.dt.date < pd.Timestamp.utcnow().date()
due_after_now = data.due_date.dt.date > pd.Timestamp.utcnow().date()

unpaid_today = data[due_after_now & created_before_now & (data.is_paid == 0)]


unpaid_today.to_csv(os.path.join(output_dir, "unpaid_today.csv"))

unpaid_today.head(20)


,var_symbol,creation_date,due_date,amount,VAT,payment_type,is_paid
ID,,,,,,,
1663900020,102726,2022-09-23 04:27:00,2022-10-07 04:27:00,11382,10,card,0
1663900560,104331,2022-09-23 04:36:00,2022-10-07 04:36:00,9076,10,card,0
1663911720,101238,2022-09-23 07:42:00,2022-10-07 07:42:00,13952,10,backward_invocing,0
1663924620,103318,2022-09-23 11:17:00,2022-10-07 11:17:00,4706,10,card,0
1663926660,100628,2022-09-23 11:51:00,2022-10-07 11:51:00,10359,10,card,0
1663936080,100110,2022-09-23 14:28:00,2022-10-07 14:28:00,3092,20,backward_invocing,0
1663954920,104421,2022-09-23 19:42:00,2022-10-07 19:42:00,14623,10,card,0
1663980780,101305,2022-09-24 02:53:00,2022-10-08 02:53:00,1289,0,pay_by_link,0
1663981620,104536,2022-09-24 03:07:00,2022-10-08 03:07:00,8952,10,pay_by_link,0
